In [143]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [144]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [145]:
code = """select A.DRVR_USER_ID, A.MARKET_NM,
B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df.head()

,DRVR_USER_ID,MARKET_NM,ACTV_DUR_MIN_QTY,ACTV_DT
0,busmom1805@gmail.com,Walmart Tulsa,0.070000000,2021-03-16
1,julieannpruitt@icloud.com,Walmart Tulsa,120.700000000,2021-03-16
2,julieannpruitt@icloud.com,Walmart Tulsa,120.700000000,2021-03-16
3,cirosalescpo@gmail.com,Walmart Tulsa,245.930000000,2021-07-10
4,cirosalescpo@gmail.com,Walmart Tulsa,245.930000000,2021-07-10


In [146]:
df = df[df["DRVR_USER_ID"].str.contains("walmart|faisal") == False]
df = df[df["ACTV_DUR_MIN_QTY"] > 0 ]
df = df.drop_duplicates()

In [154]:
code2 = """SELECT Slot_From_TS, Slot_To_TS, Spark_ZONE, DRVR_USER_ID, SLOT_HR, 
ORDER_PAYMENT, TIP_PAYMENT, SURGE_PRICE_AMT, INCENTIVE_AMT,
ACCEPTED, SPARK_DURATION
FROM `wmt-edw-sandbox.LMD_DA.SPARK_DELIVERY_DS_ALL_FINAL`
WHERE Spark_ZONE in ('Walmart Orlando', 'Walmart Tulsa', 'Walmart South Minneapolis', 'Walmart San Jose North', 'Walmart San Jose South', 'Walmart Virginia Beach') 
"""
df2 = query_bq(code2)
df2.head()

,Slot_From_TS,Slot_To_TS,Spark_ZONE,DRVR_USER_ID,SLOT_HR,ORDER_PAYMENT,TIP_PAYMENT,SURGE_PRICE_AMT,INCENTIVE_AMT,ACCEPTED,SPARK_DURATION
0,2020-11-20 14:00:00,2020-11-20 15:00:00,Walmart Virginia Beach,idjoseph.wfg@gmail.com,14,6.285000000,4.000000000,0E-9,None,1,1
1,NaT,NaT,Walmart South Minneapolis,None,8,0E-9,None,0E-9,None,<NA>,1
2,2021-09-10 18:14:15,2021-09-10 14:00:00,Walmart South Minneapolis,adixon28@live.com,18,5.314285714,None,0E-9,3.436296296,<NA>,1
3,2021-09-17 12:09:08,2021-09-17 14:00:00,Walmart South Minneapolis,None,17,0E-9,None,0E-9,None,<NA>,1
4,2021-09-17 17:55:37,2021-09-17 14:00:00,Walmart South Minneapolis,None,8,0E-9,None,0E-9,None,<NA>,1


In [155]:
df2 = df2.drop(columns = ['SLOT_HR','SPARK_DURATION'],axis = 1)

In [156]:
df2['ACTV_DT'] = df2['Slot_From_TS'].apply(lambda x: x.date())

In [157]:
df2 = pd.DataFrame(df2.groupby(['ACTV_DT','Spark_ZONE','DRVR_USER_ID','ACCEPTED'],as_index = False).sum())

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_32280\2487353495.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  df2 = pd.DataFrame(df2.groupby(['ACTV_DT','Spark_ZONE','DRVR_USER_ID','ACCEPTED'],as_index = False).sum())


In [158]:
df2 = df2.drop(columns = ['Spark_ZONE', 'ACCEPTED'],axis = 1)
df2

,ACTV_DT,DRVR_USER_ID,ORDER_PAYMENT,TIP_PAYMENT,SURGE_PRICE_AMT,INCENTIVE_AMT
0,2020-11-07,atwell.linda4@gmail.com,51.260000000,28.000000000,0E-9,48.461538462
1,2020-11-07,candismurphy84@gmail.com,11.050000000,10.000000000,0E-9,220.000000000
2,2020-11-07,cuppienail@gmail.com,11.890000000,0,0E-9,15.000000000
3,2020-11-07,fawnnaibarra@gmail.com,37.930000000,34.000000000,0E-9,46.875000000
4,2020-11-07,glenneeley2@gmail.com,9.230000000,5.000000000,0E-9,23.571428571
...,...,...,...,...,...,...
196374,2022-07-28,vgagbe@gmail.com,9.000000000,0,0E-9,0
196375,2022-07-28,williamtdad44@gmail.com,28.380000000,0,0E-9,0
196376,2022-07-28,yelhsa.n.s@gmail.com,9.000000000,0,0E-9,0
196377,2022-07-28,yodiancurry@gmail.com,50.810000000,0,0E-9,0


In [159]:
df2['total_amt'] = df2['ORDER_PAYMENT'] + df2['TIP_PAYMENT'] + df2['SURGE_PRICE_AMT'] + df2['INCENTIVE_AMT']
df2 = df2.drop(columns = ['ORDER_PAYMENT','TIP_PAYMENT','SURGE_PRICE_AMT','INCENTIVE_AMT'],axis=1)

In [162]:
df

,DRVR_USER_ID,MARKET_NM,ACTV_DUR_MIN_QTY,ACTV_DT
0,busmom1805@gmail.com,Walmart Tulsa,0.070000000,2021-03-16
1,julieannpruitt@icloud.com,Walmart Tulsa,120.700000000,2021-03-16
3,cirosalescpo@gmail.com,Walmart Tulsa,245.930000000,2021-07-10
5,fidelino18@hotmail.com,Walmart Tulsa,236.530000000,2021-07-10
7,fidelino18@hotmail.com,Walmart Tulsa,239.880000000,2021-07-10
...,...,...,...,...
1290765,aragan59@outlook.com,Walmart South Minneapolis,39.530000000,2022-06-09
1290766,aragan59@outlook.com,Walmart South Minneapolis,48.420000000,2022-06-09
1290767,aragan59@outlook.com,Walmart South Minneapolis,48.270000000,2022-06-09
1290768,aragan59@outlook.com,Walmart South Minneapolis,45.950000000,2022-06-09


In [163]:
df2

,ACTV_DT,DRVR_USER_ID,total_amt
0,2020-11-07,atwell.linda4@gmail.com,127.721538462
1,2020-11-07,candismurphy84@gmail.com,241.050000000
2,2020-11-07,cuppienail@gmail.com,26.890000000
3,2020-11-07,fawnnaibarra@gmail.com,118.805000000
4,2020-11-07,glenneeley2@gmail.com,37.801428571
...,...,...,...
196374,2022-07-28,vgagbe@gmail.com,9.000000000
196375,2022-07-28,williamtdad44@gmail.com,28.380000000
196376,2022-07-28,yelhsa.n.s@gmail.com,9.000000000
196377,2022-07-28,yodiancurry@gmail.com,50.810000000


In [167]:
df2['ACTV_DT'] = pd.to_datetime(df2['ACTV_DT'])

In [171]:
df['ACTV_DT'] = pd.to_datetime(df['ACTV_DT'])

In [176]:
td = pd.merge(df,df2,on=['DRVR_USER_ID','ACTV_DT'])

In [177]:
td = td.groupby(['ACTV_DT','MARKET_NM','DRVR_USER_ID'],as_index = False).sum()
td

,ACTV_DT,MARKET_NM,DRVR_USER_ID,ACTV_DUR_MIN_QTY,total_amt
0,2021-03-03,Walmart Virginia Beach,ajspeller@gmail.com,426.700000000,1737.750000000
1,2021-03-03,Walmart Virginia Beach,allshowbiz@gmail.com,353.900000000,573.100000000
2,2021-03-03,Walmart Virginia Beach,bmperkins155@gmail.com,428.490000000,1877.858461480
3,2021-03-03,Walmart Virginia Beach,bryantross87@gmail.com,334.280000000,870.030000000
4,2021-03-03,Walmart Virginia Beach,cmoore0830@yahoo.com,715.060000000,1096.620000014
...,...,...,...,...,...
129600,2022-07-28,Walmart Virginia Beach,vaughanfranklin54@gmail.com,249.360000000,45.690000000
129601,2022-07-28,Walmart Virginia Beach,williamtdad44@gmail.com,340.000000000,113.520000000
129602,2022-07-28,Walmart Virginia Beach,yelhsa.n.s@gmail.com,89.230000000,18.000000000
129603,2022-07-28,Walmart Virginia Beach,yodiancurry@gmail.com,568.920000000,152.430000000


In [178]:
td = td.rename(columns = {'DRVR_USER_ID': 'driver', 'ACTV_DT': 'date','ACTV_DUR_MIN_QTY':'minutes','MARKET_NM':'market'})
td

,date,market,driver,minutes,total_amt
0,2021-03-03,Walmart Virginia Beach,ajspeller@gmail.com,426.700000000,1737.750000000
1,2021-03-03,Walmart Virginia Beach,allshowbiz@gmail.com,353.900000000,573.100000000
2,2021-03-03,Walmart Virginia Beach,bmperkins155@gmail.com,428.490000000,1877.858461480
3,2021-03-03,Walmart Virginia Beach,bryantross87@gmail.com,334.280000000,870.030000000
4,2021-03-03,Walmart Virginia Beach,cmoore0830@yahoo.com,715.060000000,1096.620000014
...,...,...,...,...,...
129600,2022-07-28,Walmart Virginia Beach,vaughanfranklin54@gmail.com,249.360000000,45.690000000
129601,2022-07-28,Walmart Virginia Beach,williamtdad44@gmail.com,340.000000000,113.520000000
129602,2022-07-28,Walmart Virginia Beach,yelhsa.n.s@gmail.com,89.230000000,18.000000000
129603,2022-07-28,Walmart Virginia Beach,yodiancurry@gmail.com,568.920000000,152.430000000


In [179]:
td.to_csv('Desktop/spark-vs-payment.csv')